# Hecho por:
## EDINSON ORLANDO DORADO DORADO 1941966-3743
## ALESSANDRO DIAZ GARCIA 1940983-3743

# Este modelo de red entrena un BiLSTM + CRF para la clasificación de NER sobre el corpus Conll2002. Este modelo tiene como entrada a la red la enterización del conjunto X de entrenamiento y la enterización y categorización  de los vectores de etiquetas. En este modelo no hay matriz de embedding.

In [1]:
try:
    import seqeval
except ModuleNotFoundError as err:
    !pip install seqeval

In [2]:
!pip install tensorflow

In [3]:
import tensorflow as tf

# Instalación del paquete nltk

In [4]:
!pip install tensorflow-addons

In [5]:
!pip install keras

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from itertools import islice

import keras
#from tabulate import tabulate
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report as eskclarep
#from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
#from seqeval.metrics import classification_report as seqclarep
from sklearn.preprocessing import LabelBinarizer
from itertools import chain

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Concatenate, Lambda, Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, InputLayer, Activation, Flatten
from tensorflow.keras.optimizers import Adam, schedules
from crfta import CRF as crf4
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

from IPython.core.display import display, HTML

import datetime, os
import random

C:\Users\edins\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\edins\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of Tens

In [7]:
!pip install nltk

In [8]:
import nltk
nltk.download('conll2002')
nltk.corpus.conll2002.fileids()

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\edins\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [9]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
eval_sents = list(nltk.corpus.conll2002.iob_sents('esp.testa'))
print(len(train_sents),len(max(train_sents,key=len)))
print(len(test_sents),len(max(test_sents,key=len)))
print(len(eval_sents),len(max(eval_sents,key=len)))

8323 1238
1517 202
1915 141
CPU times: total: 1.61 s
Wall time: 1.92 s


In [10]:
print(train_sents[0])

[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


# PARTE 1. PREPROCESAMIENTO DE LOS DATOS

In [11]:
# extraen las etiquetas o labels
def sent2labels(sent):
    return [label for token, postag, label in sent]
# extraer tokens o palabras
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [12]:
sent2tokens(train_sents[0])[0]
#sent2labels(train_sents[0])[0]

'Melbourne'

In [13]:
%%time
# Conjunto de train
X_train = [sent2tokens(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

# Conjunto de testeo o de gold
X_test = [sent2tokens(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

# Conjunto de validación
X_eval = [sent2tokens(s) for s in eval_sents]
y_eval = [sent2labels(s) for s in eval_sents]

CPU times: total: 62.5 ms
Wall time: 109 ms


In [14]:
print(X_train[2])
print(y_train[2])

['El', 'Abogado', 'General', 'del', 'Estado', ',', 'Daryl', 'Williams', ',', 'subrayó', 'hoy', 'la', 'necesidad', 'de', 'tomar', 'medidas', 'para', 'proteger', 'al', 'sistema', 'judicial', 'australiano', 'frente', 'a', 'una', 'página', 'de', 'internet', 'que', 'imposibilita', 'el', 'cumplimiento', 'de', 'los', 'principios', 'básicos', 'de', 'la', 'Ley', '.']
['O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O']


In [15]:
import numpy as np

words, tagsss = set([]), set([])

for s in (X_train + X_eval + X_test):
    for w in s:
        words.add(w.lower())

for ts in (y_train + y_eval + y_test):
    for t in ts:
        tagsss.add(t)

word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs

tag2index = {t: i + 2 for i, t in enumerate(list(tagsss))}
tag2index['-PAD-'] = 0  # The special value used to padding
tag2index['-OOV-'] = 1  # The special value used to padding

print (len(word2index))
print (len(tag2index))
print(tag2index)


print(tagsss)

28384
11
{'B-PER': 2, 'B-MISC': 3, 'I-ORG': 4, 'I-PER': 5, 'B-ORG': 6, 'I-LOC': 7, 'O': 8, 'B-LOC': 9, 'I-MISC': 10, '-PAD-': 0, '-OOV-': 1}
{'B-PER', 'B-MISC', 'I-ORG', 'I-PER', 'B-ORG', 'I-LOC', 'O', 'B-LOC', 'I-MISC'}


In [16]:
train_sentences_X, eval_sentences_X, test_sentences_X, train_tags_y, eval_tags_y, test_tags_y = [], [], [], [], [], []

for s in X_train:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    train_sentences_X.append(s_int)

for s in X_eval:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    eval_sentences_X.append(s_int)

for s in X_test:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    test_sentences_X.append(s_int)

for s in y_train:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    train_tags_y.append(s_int)

for s in y_eval:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    eval_tags_y.append(s_int)

for s in y_test:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    test_tags_y.append(s_int)

In [17]:
# En esta parte se enterizan las sentencias de train, val y testeo con longitudes diferentes.

In [18]:
print("Longitudes de las Matrices:")
print(len(train_sentences_X))
print(len(eval_sentences_X))
print(len( test_sentences_X))
print(len(train_tags_y))
print(len(eval_tags_y))
print(len(test_tags_y))

print("\nMuestra de Datos presentes en las Matrices con las transformaciones:\n")


print(train_sentences_X[0])
print(eval_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(eval_tags_y[0])
print(test_tags_y[0])


Longitudes de las Matrices:
8323
1915
1517
8323
1915
1517

Muestra de Datos presentes en las Matrices con las transformaciones:

[3006, 307, 24226, 7807, 1011, 24171, 9418, 307, 24101, 7807, 12880]
[18139, 7122, 307, 20600, 7807, 1011, 16003, 9418, 307, 7444, 7807, 12880]
[13428, 2386, 1011, 16003, 9418, 307, 7444, 7807, 12880]
[9, 8, 9, 8, 8, 8, 8, 8, 6, 8, 8]
[9, 7, 8, 9, 8, 8, 8, 8, 8, 6, 8, 8]
[9, 7, 8, 8, 8, 8, 6, 8, 8]


# Se procede a Normalizar las matrices con la longitud de la columna=MAX_LENGTH1 para que todas contengan el mismo numero de columnas, con la longitud máxima de palabras encontradas anteriormente y se agregan ceros a la derecha en las posiciones que hacen falta en el vector.

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LENGTH=202
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
eval_sentences_X = pad_sequences(eval_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
eval_tags_y = pad_sequences(eval_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')

print(train_sentences_X[0])
print(train_sentences_X.shape)
print(eval_sentences_X[0])
print(eval_sentences_X.shape)
print(test_sentences_X[0])
print(test_sentences_X.shape)
print(train_tags_y[0])
print(train_tags_y.shape)
print(eval_tags_y[0])
print(eval_tags_y.shape)
print(test_tags_y[0])
print(test_tags_y.shape)

[ 3006   307 24226  7807  1011 24171  9418   307 24101  7807 12880     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [20]:
def to_categoricals(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [21]:
def encode(data):
    print('Shape of data (BEFORE encode): %s' % str(data.shape))
    encoded = to_categorical(data)
    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
    return encoded

# Se realiza la categorización one-hot de las etiquetas o labels de entrenamiento, testeo y validación

In [22]:
cat_train_tags_y = to_categoricals(train_tags_y, len(tag2index))
cat_eval_tags_y  = to_categoricals(eval_tags_y, len(tag2index))
cat_test_tags_y  = to_categoricals(test_tags_y, len(tag2index))

print(cat_train_tags_y[1])
print(len(cat_train_tags_y))
print(cat_train_tags_y.shape)
print(len(cat_test_tags_y))

[[0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
8323
(8323, 202, 11)
1517


# Carga del embeddings de Europarl
En la variable **embedding_matrix** se guarda el conjunto de vectores embebidos

In [23]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
EMBED_DIM=300
file = 'word2vec_skip-gram_model_300.txt'
file= 'Europarl_fasttext_skip_model_300.txt'
embedding_matrix = bme(file, len(word2index), EMBED_DIM, word2index)

Cargando archivo...


179359it [00:21, 8303.31it/s]


Encontrado 179359 Word Vectors.


100%|████████████████████████████████████████████████████████████████████████| 28384/28384 [00:00<00:00, 323261.65it/s]

Convertidos: 18312 Tokens | Perdidos: 10365 Tokens


# 2. MODELO DE LA RED NEURONAL RECURSIVA (BILSTM+ CRF)

#  Modelo matématico
# Aqui  se define el modelo matémático de la  máquina de aprendizaje que en este caso en una red neuronal recursiva RNN. Específicamente es un Bilstm que  es un Lstm en dos direcciones  y un CRF  para mejorar la distribución probabilística producto de la  red neunoral. En esta red de prueba una estructura de Masking para mejorar las distribuciones probabilídticas. En pocas palabras cumple la función de un distributed. Esta red neuronal tiene una entrada **input = Input(shape=(MAX_LENGTH,))**  esta entrada tiene la longitud de la máxima sentencia  que es 202. Luego se define la matriz de embedding que es una vectorización de palabras usando Word2vec. Esta es una matriz que se vuelve de tres dimensiones 28384 X 202 X 300 y cuando se tiene embedding de palabras se llena son los pesos de la matriz preentrenada, ya sea de word2vec, fasttext o glove. También se pueden concatenar matrices de embedding de otras características lingúísticas como el POS, lema, ect. Luego viene la

In [24]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model

In [25]:
from tf2crf import CRF as crf6
import keras as k
from mwrapper import ModelWithCRFLoss, ModelWithCRFLossDSCLoss
from keras.layers import Masking

input = Input(shape=(MAX_LENGTH,))
word_embedding_size = 300
EMBED_DIM = 300
# Embedding Layer
model = Embedding(input_dim=len(word2index),
                  weights=[embedding_matrix],  # Initializing with FastText embeddings
                output_dim=word_embedding_size,
                input_length=MAX_LENGTH,
                mask_zero=False)(input)
model = Bidirectional(LSTM(units=50,
                     return_sequences=True,
                     dropout=0.5,
                     recurrent_dropout=0))(model)

model  = Dropout(0.5, name='dropout_lstm')(model)
model  = Dense(units=EMBED_DIM, activation='tanh')(model)
model  = Dense(units=len(tag2index), activation='tanh')(model)

model = Masking(mask_value= 0.,input_shape=(MAX_LENGTH,len(tag2index)))(model)

crf = crf6(units=len(tag2index), name="ner_crf")
predictions = crf(model)

base_model = Model(inputs=input, outputs=predictions)
model = ModelWithCRFLoss(base_model, sparse_target=True)
#keras.mixed_precision.set_global_policy("mixed_float16")
model.compile(optimizer='adam')

In [26]:
#with tf.device('/device:GPU:0'):
history= model.fit(train_sentences_X, cat_train_tags_y,
      validation_data=(eval_sentences_X, cat_eval_tags_y),
      batch_size=128,
      epochs=80,
      verbose=2)

Epoch 1/80
66/66 - 40s - loss: 135.2910 - accuracy: 0.8733 - val_loss_val: 51.8831 - val_val_accuracy: 0.9780 - 40s/epoch - 611ms/step
Epoch 2/80
66/66 - 30s - loss: 41.8735 - accuracy: 0.9799 - val_loss_val: 32.3108 - val_val_accuracy: 0.9803 - 30s/epoch - 450ms/step
Epoch 3/80
66/66 - 29s - loss: 28.7951 - accuracy: 0.9805 - val_loss_val: 24.5882 - val_val_accuracy: 0.9804 - 29s/epoch - 444ms/step
Epoch 4/80
66/66 - 30s - loss: 22.9458 - accuracy: 0.9805 - val_loss_val: 20.7450 - val_val_accuracy: 0.9804 - 30s/epoch - 453ms/step
Epoch 5/80
66/66 - 30s - loss: 19.6816 - accuracy: 0.9806 - val_loss_val: 18.2488 - val_val_accuracy: 0.9804 - 30s/epoch - 459ms/step
Epoch 6/80
66/66 - 34s - loss: 17.1915 - accuracy: 0.9806 - val_loss_val: 16.6215 - val_val_accuracy: 0.9804 - 34s/epoch - 510ms/step
Epoch 7/80
66/66 - 33s - loss: 15.7055 - accuracy: 0.9806 - val_loss_val: 15.4195 - val_val_accuracy: 0.9805 - 33s/epoch - 498ms/step
Epoch 8/80
66/66 - 31s - loss: 14.4485 - accuracy: 0.9807 - v

In [27]:
print(tag2index)
print(test_sentences_X)
y_pred= model.predict(test_sentences_X)
print(y_pred.shape)
print(y_pred)

{'B-PER': 2, 'B-MISC': 3, 'I-ORG': 4, 'I-PER': 5, 'B-ORG': 6, 'I-LOC': 7, 'O': 8, 'B-LOC': 9, 'I-MISC': 10, '-PAD-': 0, '-OOV-': 1}
[[13428  2386  1011 ...     0     0     0]
 [12278     0     0 ...     0     0     0]
 [ 2670   957  8006 ...     0     0     0]
 ...
 [13428 20829 20191 ...     0     0     0]
 [ 1373 21611   642 ...     0     0     0]
 [13428 24340 10415 ...     0     0     0]]
48/48 [==============================] - 3s 42ms/step
(1517, 202)
[[9 7 8 ... 0 0 0]
 [8 0 0 ... 0 0 0]
 [8 8 8 ... 0 0 0]
 ...
 [8 9 8 ... 0 0 0]
 [8 8 8 ... 0 0 0]
 [8 8 8 ... 0 0 0]]


In [28]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
index2tag = {i: t for t, i in tag2index.items()}
print(index2tag)
y1_pred = logits_to_tokens(y_pred, index2tag)
print(y1_pred[10])

{2: 'B-PER', 3: 'B-MISC', 4: 'I-ORG', 5: 'I-PER', 6: 'B-ORG', 7: 'I-LOC', 8: 'O', 9: 'B-LOC', 10: 'I-MISC', 0: '-PAD-', 1: '-OOV-'}
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PA

In [29]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
index2tag = {i: t for t, i in tag2index.items()}
print(index2tag)
y1_true = logits_to_tokens(test_tags_y, index2tag)
print(y1_true[10])

{2: 'B-PER', 3: 'B-MISC', 4: 'I-ORG', 5: 'I-PER', 6: 'B-ORG', 7: 'I-LOC', 8: 'O', 9: 'B-LOC', 10: 'I-MISC', 0: '-PAD-', 1: '-OOV-'}
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-'

In [30]:
#hh1 = seqclarep(results['Expected'], results['Predicted'])
#print('\nclassification_report:\n', hh1)
from seqeval.metrics import classification_report as seqclarep
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
print("precision: {:.1%}".format(precision_score(y1_true, y1_pred)))
print("   recall: {:.1%}".format(recall_score(y1_true,    y1_pred)))
#print(" accuracy: {:.1%}".format(accuracy_score(y1_true,  y1_pred)))
print(" F1-score: {:.1%}".format(f1_score(y1_true,        y1_pred)))

C:\Users\edins\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: -PAD- seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


precision: 68.3%
   recall: 68.4%
 F1-score: 68.4%


In [31]:
import pandas as pd
li1 = sum(y1_true, [])
li2 = sum(y1_pred, [])

results = pd.DataFrame(columns=['Expected', 'Predicted'])

results['Expected'] = li1
results['Predicted'] = li2

In [32]:
from sklearn.metrics import classification_report as eskclarep
report = eskclarep(results['Expected'], results['Predicted'])
#print('\nclassification_report:\n', report)

print(report_to_df(report))

  Class Name precision recall f1-score support
0      -PAD-      1.00   1.00     1.00  254901
1      B-LOC      0.77   0.75     0.76    1084
2     B-MISC      0.38   0.41     0.39     339
3      B-ORG      0.72   0.75     0.73    1400
4      B-PER      0.87   0.72     0.79     735
5      I-LOC      0.75   0.55     0.63     325
6     I-MISC      0.53   0.33     0.41     557
7      I-ORG      0.74   0.61     0.67    1104
8      I-PER      0.90   0.79     0.84     634
9          O      0.97   0.99     0.98   45355


In [33]:
test_samples = [
    "La Federación nacional de cafeteros de Colombia es una entidad del estado , creada en los años 70’s .".split(), 
    "El primer presidente el Dr Augusto Guerra contó con el aval de la Asociación Colombiana de Aviación".split(),
    "En varias ciudades colombianas ; Cali , medellín , corozal funciona la entidad .".split(),
    "Estas personas viene del instituto colombiano del café ubicado en la calle Cali .".split()
]
#print(max(test_samples))
print(test_samples)

[['La', 'Federación', 'nacional', 'de', 'cafeteros', 'de', 'Colombia', 'es', 'una', 'entidad', 'del', 'estado', ',', 'creada', 'en', 'los', 'años', '70’s', '.'], ['El', 'primer', 'presidente', 'el', 'Dr', 'Augusto', 'Guerra', 'contó', 'con', 'el', 'aval', 'de', 'la', 'Asociación', 'Colombiana', 'de', 'Aviación'], ['En', 'varias', 'ciudades', 'colombianas', ';', 'Cali', ',', 'medellín', ',', 'corozal', 'funciona', 'la', 'entidad', '.'], ['Estas', 'personas', 'viene', 'del', 'instituto', 'colombiano', 'del', 'café', 'ubicado', 'en', 'la', 'calle', 'Cali', '.']]


In [34]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)

test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)
print(test_samples_X.shape)

[[13428  7629  6654 13976   268 13976 20391 25015 26324  9516 24726 11504
   1011 11207 13349  1373  1392     1 12880     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [35]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

1/1 [==============================] - 0s 47ms/step
[[8 6 4 4 4 4 4 8 8 8 8 6 8 8 8 8 8 6 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [8 8 8 8 2 5 5 8 8 8 8 8 8 8 8 8 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [8 8 8 8 8 9 8 9 8 8 8 8 8 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [36]:
#print(len(predictions))
log_tokens = logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})
print(log_tokens)

[['O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-'

In [37]:
!pip install tabulate
from tabulate import tabulate

heads1 = test_samples[0]
body1 = [log_tokens[0][:len(test_samples[0])]]

heads2 = test_samples[1]
body2 = [log_tokens[1][:len(test_samples[1])]]

heads3 = test_samples[2]
body3 = [log_tokens[2][:len(test_samples[2])]]

heads4 = test_samples[3]
body4 = [log_tokens[3][:len(test_samples[3])]]

print(tabulate(body1, headers=heads1))

print ("\n")

print(tabulate(body2, headers=heads2))

print ("\n")

print(tabulate(body3, headers=heads3))

print ("\n")

print(tabulate(body4, headers=heads4))

La    Federación    nacional    de     cafeteros    de     Colombia    es    una    entidad    del    estado    ,    creada    en    los    años    70’s    .
----  ------------  ----------  -----  -----------  -----  ----------  ----  -----  ---------  -----  --------  ---  --------  ----  -----  ------  ------  ---
O     B-ORG         I-ORG       I-ORG  I-ORG        I-ORG  I-ORG       O     O      O          O      B-ORG     O    O         O     O      O       B-ORG   O


El    primer    presidente    el    Dr     Augusto    Guerra    contó    con    el    aval    de    la    Asociación    Colombiana    de    Aviación
----  --------  ------------  ----  -----  ---------  --------  -------  -----  ----  ------  ----  ----  ------------  ------------  ----  ----------
O     O         O             O     B-PER  I-PER      I-PER     O        O      O     O       O     O     O             O             O     O


En    varias    ciudades    colombianas    ;    Cali    ,    medellín    ,    